In [53]:
import re
import os
import pandas as pd

def load_fasta(fasta_path):
    """Without Using Biopython"""
    fasta = open(fasta_path, 'r').read()
    record_dict = dict()
    records = fasta.split('>')
    for rec in records:
        if rec:
            rec_lines = rec.split('\n')
            header = rec_lines[0]
            seq = ''.join(rec_lines[1:]).replace(' ', '')
            record_dict[header] = seq
    return record_dict

def check_seq_for_motif(seq):
    """
    Check seq for motif
    Input:
    seq: str, sequence
    motif: str, motif to search seq for
    """
    pattern = '(AT){3,}'
    r_all = re.findall(pattern, seq)
    r_first = re.search(pattern, seq)
    at_count = len(r_all)
    if r_first:
        start, end = r_first.span()
        print(f'AT repeat discovered {at_count} time(s)\n')
        print(f'AT repeat first found at positions {start} - {end}')

    else:
        print(f'Unable to find AT repeat\n\n')
        
    return at_count
        
        
def check_all_seqs(fasta_path):
    """
    Input:
    fasta_path: str, path to fasta file
    motif: str, motif to search for within fasta
    
    """
    if os.path.exists(fasta_path):
        record_dict = load_fasta(fasta_path)
    else:
        print(f'Fasta path {fasta_path} does not exist.')
        raise OSError
    total_count = 0
    N_records = 0
    for header in record_dict:
        N_records += 1
        print(f'Fasta header: {header}\n')
        tcount = check_seq_for_motif(record_dict[header])
        total_count += tcount
    print(f'AT repeat discovered {total_count} time(s) across {N_records} records.\n\n\n')

    
def parse_embl(embl_path, write=True):
    embl = open(embl_path, 'r').read()
    embl_records = embl.split('//')
    embl_dict = dict()
    for record in embl_records[:-1]:

        embl_lines = record.split('\n')
        seq_id = None
        description = None
        seq_start = None
        for ind, line in enumerate(embl_lines):
            if line[:2] == 'ID':
                seq_id = line[2:].strip()
            elif line[:2] == 'DE':
                description = line[2:].strip()
            elif line[:2] == 'SQ':
                seq_start = ind + 1
                break
        sequence = ''.join([line for line in embl_lines[seq_start:]]).replace(' ', '')
        sequence = re.sub('[0-9]+', '', sequence).upper()
        embl_dict[seq_id] = {'descrption': description, 'sequence': sequence}
    embl_df = pd.DataFrame(embl_dict).T.reset_index()
    embl_df.rename(columns={'index': 'Seq_ID'}, inplace=True)
    if write is True:
        embl_df.to_csv('embl_info.csv')
    return embl_df
        

check_all_seqs('Test.txt')
embl_df = parse_embl('EMBL_records.txt')

embl_df


Fasta header: TestSeq

AT repeat discovered 1 time(s)

AT repeat first found at positions 89 - 105
AT repeat discovered 1 time(s) across 1 records.





,Seq_ID,descrption,sequence
0,M91373; SV 1; linear; mRNA; STD; PLN; 1131 BP.,"Cucumis sativus peroxidase mRNA, complete cds.",ACCAGAGAAGACCCCATTTGCAGTATCAAAAATGGGTTTACCTAAA...
1,M57705; SV 1; linear; mRNA; STD; ROD; 237 BP.,"Rat truncated thyroid peroxidase mRNA, 3' end.",CATCGATCATGACATTGCTCTCACACCACAGAGCACCAGCACAGCA...


# Using a Custom wrapper through the sh library 
### Only available in unix-like systems

In [4]:

from sh import blastn
from Bio.Blast.Applications import NcbiblastnCommandline as bioblastn
from Bio import SearchIO
import pandas as pd


def blastn_wrapper(query, out, db='protein', task='megablast', perc_identity=92, qcov_hsp_perc=100,
                           max_target_seqs=15, threads=60, buffer=False, word_size=15):

    """
    Input:
    query: str, path to fasta file of universal regions
    out: str, path to output blast tab file
    perc_identity: int, percentage identical to query entries
    qcov_hsp_perc: int, percentage of query entry covered by hits
    max_target_seqs: int, number of hits to show in tab file
    Used for specificity analysis of final universal regions.
    """

    if not db:
        db = 'protein'

    outfmt = "6" # qseqid sseqid pident qcovhsp length mismatch gapopen qstart qend sstart send evalue bitscore"

    blastn('-db', db, '-perc_identity', perc_identity, '-task', task,
               '-qcov_hsp_perc', qcov_hsp_perc, '-dust', 'no', '-max_target_seqs', max_target_seqs,
               '-out', out, '-query', query, '-outfmt', outfmt, '-word_size', word_size, '-gapopen', 5, '-gapextend', 2,
            '-evalue', 10)

fasta_path = 'mitochondrial.fasta'
blast_output = 'mitochondrial.tab'
blastdb_path = '/home/pradeep/blastdb/nt'
blastn_wrapper(fasta_path, blast_output, db = blastdb_path)



KeyboardInterrupt: 

# Using Biopythons Blast Wrapper

In [5]:
blast_txt = 'mitochondrial.txt'
blast_cline = bioblastn(query=fasta_path, out=blast_txt, task='megablast', remote=True, db='nt',)
stdout, stderr = blast_cline()
print(blast_cline)

blastn -out mitochondrial.txt -query mitochondrial.fasta -db nt -remote -task megablast


# Parsing using biopython

In [13]:
bltab = SearchIO.parse('mitochondrial.txt', 'blast-text')
for line in bltab:
    print(line)

Program: blastn (2.10.1+)
  Query: NM_001319878.1 (788)
         Camelus dromedarius superoxide dismutase 2 (SOD2),mRNA; nuclear gene...
 Target: Nucleotide collection (nt)
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  NM_001319878.1  Camelus dromedarius superoxide dismutas...
            1      1  XM_032485290.1  PREDICTED: Camelus ferus superoxide dis...
            2      1  XM_010965766.1  PREDICTED: Camelus bactrianus superoxid...
            3      1  XM_015246572.2  PREDICTED: Vicugna pacos superoxide dis...
Program: blastn (2.10.1+)
  Query: XM_003872801.1 (490)
         Leishmania mexicana MHOM/GT/2001/U1103 mitochondrialpartial mRNA
 Target: Nucleotide collection (nt)
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -

/home/alarsen/miniconda3/envs/dtra/lib/python3.7/site-packages/Bio/SearchIO/_legacy/NCBIStandalone.py:93: BiopythonWarning: Parsing BLAST plain text output file is not a well supported functionality anymore. Consider generating your BLAST output for parsing as XML or tabular format instead.
  BiopythonWarning,


# Parsing using Pandas

In [9]:
headers = ['query', 'subject', 'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
                   'query_start', 'query_end', 'subject_start', 'subject_end', 'e_value', 'bitscore']
df = pd.read_csv('mitochondrial.tab', '\t', skiprows=7, header=None)
df.columns = headers
df

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,XM_001463625.1,CP018576.1,100.000,560,0,0,1,560,43332,43891,0.0,1077
1,XM_001463625.1,XM_003858804.1,100.000,560,0,0,1,560,1,560,0.0,1077
2,XM_001463625.1,FR799597.2,100.000,560,0,0,1,560,43331,43890,0.0,1077
3,XM_001463625.1,CP048173.1,100.000,560,0,0,1,560,44827,45386,0.0,1077
4,XM_001463625.1,LR812943.1,100.000,560,0,0,1,560,56335,56894,0.0,1077
5,XM_001463625.1,LR812630.1,100.000,560,0,0,1,560,56281,56840,0.0,1077
6,XM_001463625.1,FR796442.1,100.000,560,0,0,1,560,44829,45388,0.0,1077
7,XM_001463625.1,XM_001463625.1,100.000,560,0,0,1,560,1,560,0.0,1077
8,XM_001463625.1,CP029509.1,99.821,560,1,0,1,560,56327,56886,0.0,1071
9,XM_001463625.1,CP048137.1,99.821,560,1,0,1,560,44821,45380,0.0,1071
